In [1]:
import skl2onnx
import onnx
import sklearn
import numpy as np
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
features_df = pd.read_csv('features.csv')

X_train, X_val, y_train, y_val = train_test_split(features_df [['power','pitch_mean','pitch_sd','voiced_fr']], features_df['label'], test_size=0.3)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((583, 4), (251, 4), (583,), (251,))

In [ ]:
mean = X_train.mean(0)
sd =  X_train.std(0)

X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

model  = RandomForestClassifier(max_depth=8, random_state=0)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

training_acc = np.mean(yt_p==y_train)
validation_acc = np.mean(yv_p==y_val)

print('Training Accuracy', training_acc)
print('Validation  Accuracy', validation_acc)



In [ ]:
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onx = convert_sklearn(model, initial_types=initial_type,
                      target_opset=12)

with open("RF_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [3]:
sess = rt.InferenceSession("RF_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name],
                    {input_name: X_val.to_numpy().astype(np.float32)})[0]

In [4]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, pred_onx)

0.5099601593625498

In [ ]:
X_val.to_numpy()

In [5]:
type(pred_onx.tolist())

list

In [19]:
X_dp = pd.DataFrame([0.195328529, 399.2141388, 34.43785081, 0.686934023]).T


sess = rt.InferenceSession("RF_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name],
                    {input_name: X_dp.to_numpy().astype(np.float32)})[0]

In [20]:
pred_onx

array(['Starwars'], dtype=object)